In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-xsum")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'A New York woman has pleaded not guilty to falsely claiming to be married 10 times, including to eight men from different countries, in what prosecutors say was an immigration scam.'}]


In [2]:
from datasets import load_dataset

dataset = load_dataset("xsum")

Found cached dataset xsum (C:/Users/srina/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [4]:
num_rows_to_keep = 5000

dataset['train'] = dataset['train'].shuffle(seed=42).select([i for i in range(num_rows_to_keep)])
dataset['validation'] = dataset['validation'].shuffle(seed=42).select([i for i in range(num_rows_to_keep)])
dataset['test'] = dataset['test'].shuffle(seed=42).select([i for i in range(num_rows_to_keep)])

Loading cached shuffled indices for dataset at C:\Users\srina\.cache\huggingface\datasets\xsum\default\1.2.0\082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71\cache-0179b5b9071d9bab.arrow
Loading cached shuffled indices for dataset at C:\Users\srina\.cache\huggingface\datasets\xsum\default\1.2.0\082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71\cache-dd5e4180a7940dfa.arrow
Loading cached shuffled indices for dataset at C:\Users\srina\.cache\huggingface\datasets\xsum\default\1.2.0\082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71\cache-fb8b8c7a3545eb60.arrow


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 5000
    })
})

In [6]:
text = dataset['train']['document'][0]
print(text)

In Wales, councils are responsible for funding and overseeing schools.
But in England, Mr Osborne's plan will mean local authorities will cease to have a role in providing education.
Academies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.
It is a significant development in the continued divergence of schools systems on either side of Offa's Dyke.
And although the Welsh Government will get extra cash to match the money for English schools to extend the school day, it can spend it on any devolved policy area.
Ministers have no plans to follow suit.
At the moment, governing bodies are responsible for setting school hours and they need ministerial permission to make significant changes.
There are already more than 2,000 secondary academies in England and its extension to all state schools is unlikely to shake the Welsh Government's attachment to what they call a "community, comprehensive model" for scho

In [7]:
dataset['train']['summary'][0]

'As Chancellor George Osborne announced all English state schools will become academies, the Welsh Government continues to reject the model here.'

In [8]:
ARTICLE = dataset['train']['document'][0]
summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
print(summary)

Chancellor George Osborne's announcement that all English state schools will become academies has been described as "scandalous" by NUT Cymru.


In [9]:
from flair.data import Sentence
from flair.models import SequenceTagger
from segtok.segmenter import split_single

def NER(summary, text):
    tagger = SequenceTagger.load('ner-ontonotes')

    sentence = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]
    tagger.predict(sentence)
    text_NER = []
    for sent in sentence:
        for entity in sent.get_spans('ner'):
            text_NER.append((entity.text.strip(), entity.tag))

    sentence = [Sentence(sent, use_tokenizer=True) for sent in split_single(summary)]
    tagger.predict(sentence)
    sum_NER = []
    for sent in sentence:
        for entity in sent.get_spans('ner'):
            sum_NER.append((entity.text.strip(), entity.tag))
    return sum_NER, text_NER

In [10]:
sum_NER, text_NER = NER(summary, text)
print(sum_NER)
print("\n")
print(text_NER)

2023-09-26 17:49:02,651 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
[('George Osborne', 'PERSON'), ('English', 'NORP'), ('NUT Cymru', 'ORG')]


[('Wales', 'GPE'), ('England', 'GPE'), ('Osborne', 'PERSON'), ("Offa's Dyke", 'ORG'), ('the Welsh Government', 'ORG'), ('English', 'LANGUAGE'), ('the school day', 'DATE'), ('school hours', 'TIME'), ('more than 2,000', 'CARDINAL'), ('England', 'GPE'), ("the Welsh Government's", 'ORG'), ('Scotland', 'GPE'), ('UK', 'GPE'), ('Huw Lewis', 'PERSON'), ('Wales', 'GPE'), ('Tories', 'NORP'), ('hu

In [11]:
len(dataset['train'])

5000

In [12]:
summaries = []
train_summaries = []
summaries_NER = []
train_NER = []
for i in range(0,len(dataset['train'])):
    ARTICLE = dataset['train']['document'][i]
    ARTICLE = ARTICLE[:1024]
    summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
    summaries.append(summary)
    train_summaries.append(dataset['train']['summary'][i])
    
    sum_NER, text_NER = NER(summary, ARTICLE)
    summaries_NER.append(sum_NER)
    train_NER.append(text_NER)

2023-09-26 17:49:17,214 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-26 17:49:31,182 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

Your max_length is set to 130, but you input_length is only 86. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


2023-09-26 17:51:54,809 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-26 17:52:05,244 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

Your max_length is set to 130, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


2023-09-26 17:52:42,556 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-26 17:52:54,870 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

...

2023-09-27 11:52:56,534 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC


Your max_length is set to 130, but you input_length is only 85. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


2023-09-27 11:53:08,212 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-27 11:53:20,135 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

Your max_length is set to 130, but you input_length is only 75. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


2023-09-27 11:55:15,936 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
2023-09-27 11:55:26,692 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TI

In [13]:
import pandas as pd
data = {'Original Text': train_summaries, 'NER for Original': train_NER, 'Generated Summaries': summaries, 'NER for Generated': summaries_NER}
df = pd.DataFrame(data)

In [15]:
df.to_csv('Summaries_NER.csv', index = False)

In [16]:
df

,Original Text,NER for Original,Generated Summaries,NER for Generated
0,As Chancellor George Osborne announced all Eng...,"[(Wales, GPE), (England, GPE), (Osborne, PERSO...",The chancellor's plan to extend the school day...,"[(England, GPE), (Wales, GPE)]"
1,Discounted bus tickets for 16 to 18 year olds ...,"[(Up to 100,000, CARDINAL), (The Young Persons...",Young people in Greater Manchester are to get ...,"[(Greater Manchester, GPE)]"
2,"""The winner takes it all, the loser has to fal...","[(Middlesbrough, GPE), (Brighton, GPE), (Satur...",The final day of the Championship season has t...,"[(Championship, EVENT), (one, CARDINAL)]"
3,Support for vulnerable children in England is ...,"[(35%, PERCENT), (five years, DATE), (Barnardo...",Hundreds of children's centres in England are ...,"[(Hundreds, CARDINAL), (England, GPE)]"
4,The Philippines has submitted evidence to a UN...,"[(China, GPE), (a day, DATE), (Philippine, NOR...","The Philippines has submitted more than 4,000 ...","[(Philippines, GPE), (more than 4,000, CARDINA..."
...,...,...,...,...
4995,A man thought to have been murdered in Oxford ...,"[(Jack Phipps, PERSON), (53, DATE), (22 June, ...",The death of a man who was stabbed in Oxford i...,"[(Oxford, GPE)]"
4996,Women's Super League Two side Tottenham Ladies...,"[(Wiltshire, ORG), (27, CARDINAL), (45, CARDIN...",Women's Super League One side Tottenham Hotspu...,"[(Super League One, ORG), (Tottenham Hotspur, ..."
4997,Burundi's government has promised to crack dow...,"[(Friday, DATE), (three, CARDINAL), (two, CARD...",Burundi's President Pierre Nkurunziza has vowe...,"[(Burundi, GPE), (Pierre Nkurunziza, PERSON)]"
4998,Swansea boss Paul Clement is determined to kee...,"[(Gylfi Sigurdsson, PERSON), (Fernando Llorent...",Swansea City head coach Paul Clement says ther...,"[(Swansea City, GPE), (Paul Clement, PERSON), ..."


In [ ]:
import string
def frequencies(word, text):
    count = 0
    word = word.lower()
    text = text.lower()
    
    translator = str.maketrans('', '', string.punctuation+'\\')

    text = text.translate(translator)
    word = word.translate(translator)
    n = len(word.split())  # Get the length of the n-gram
    if n == 1:
        for i in text.split():
            if i == word:
                count += 1
    else:
        words = text.split()
        for i in range(len(words) - n + 1):
            ngram = " ".join(words[i:i + n])
            if ngram == word:
                count += 1
    return count

In [ ]:
import nltk
import numpy as np
from nltk.probability import FreqDist

def returnDists(NER_list1, NER_list2, text1, text2):
    word_freq1 = []
    total_words1 = len(text1.split())
    for i in NER_list1:
        word_freq1.append((i, float(frequencies(i,text1)/total_words1)))
    word_probabilities1 = np.array(word_freq1)
    
    word_freq2 = []
    total_words2 = len(text2.split())
    for i in NER_list2:
        word_freq2.append((i, float(frequencies(i,text2)/total_words2)))
    word_probabilities2 = np.array(word_freq2)

    # Display the two probability distributions
    print("Probability Distribution for Text 1:")
    print(word_probabilities1)

    print("\nProbability Distribution for Text 2:")
    print(word_probabilities2)
    
    return word_probabilities1, word_probabilities2


In [ ]:
def kl(dist1, dist2):
    return np.sum(dist1 * np.log(dist1 / dist2))

In [ ]:
entities_sum = []
for i in sum_NER:
    if i[0] not in entities_sum:
        entities_sum.append(i[0])
    else:
        continue

In [ ]:
entities_text = []
for i in text_NER:
    if i[0] not in entities_text:
        entities_text.append(i[0])
    else:
        continue

In [ ]:
for i in range(0,len(entities_sum)):
    if i >= len(entities_sum):
        break
    if entities_sum[i] not in entities_text:
        entities_sum.remove(entities_sum[i])
        i -= 1

for i in range(0,len(entities_text)):
    if entities_text[i] not in entities_sum:
        entities_sum.append(entities_text[i])

In [ ]:
print(sum_NER, entities_sum)
print(text_NER, entities_text)

In [ ]:
print(text)

In [ ]:
d1, d2 = returnDists(entities_sum, entities_text, summary, text)

In [ ]:
dist1 = []
for i in range(0,len(d1)):
    temp = float(d1[i][1])
    dist1.append([d1[i][0], temp])

In [ ]:
dist2 = []
for i in range(0,len(d2)):
    temp = float(d2[i][1])
    dist2.append([d2[i][0], temp])

In [ ]:
dist1, dist2

In [ ]:
# Sort both distributions based on the words (first element of each tuple)
dist1 = sorted(dist1, key=lambda x: x[0])
dist2 = sorted(dist2, key=lambda x: x[0])

In [ ]:
for i in range(0,len(dist2)):
    if i>=len(dist2):
        break
    if dist2[i][1] == 0.0:
        dist1 = np.delete(dist1,i, axis = 0)
        dist2 = np.delete(dist2,i, axis = 0)
        i -= 1

In [ ]:
dist1_keys = dist1
dist2_keys = dist2

In [ ]:
for i in range(0,len(dist1)):
    dist1[i] = dist1[i][1]
for i in range(0,len(dist2)):
    dist2[i] = dist2[i][1]

In [ ]:
from scipy.special import rel_entr

sum(rel_entr(dist1, dist2))

In [ ]:
d1, d2

In [ ]:
"""original = []
summaries = []
ner_original = []
ner_summary = []
for i in range(0,len(dataset['train'])):
    print(i)
    original.append(dataset['train']['document'][i])
    ARTICLE = dataset['train']['document'][i]
    summary = list(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)[0].values())[0]
    summaries.append(summary)
    print("Summaries done")
    sum_NER, text_NER = NER(summary)
    ner_original.append(text_NER)
    ner_summary.append(sum_NER)
    print("NER done")"""